In [1]:
# !pip install -U tensorflow tensorflow-hub

In [2]:
# 依赖：
# !pip install sentencepiece
# !pip install jieba
# !pip install regex
# !pip install tensorflow
# !pip install tensorflow-hub

In [3]:
import tensorflow_hub as hub
import tensorflow as tf

from gpt2_tokenizer import GPT2Tokenizer

In [4]:
print(tf.__version__)

2.3.1


In [5]:
print(hub.__version__)

0.10.0


In [6]:
tokenizer = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [7]:
gpt = hub.load('./cpm-lm-tf2_v2/')

In [8]:
def sample(tokenizer, gpt, sentence, number=1, length=20, top_p=0.9, temperature=0.9):
    """
    numbert: 输出句子个数
    length: 输出最大长度
    top_p: token的概率排在这以上才有效
    temperature: 温度
    """
    inputs = tf.constant([tokenizer.encode(sentence)] * number, dtype=tf.int64)
    length = tf.constant(length, dtype=tf.int64)
    ret = gpt.signatures['serving_default'](
        inp=inputs,
        length=length,
        top_p=tf.constant(top_p, tf.float32),
        temperature=tf.constant(temperature, tf.float32)
    )['output_0']
    return [
        tokenizer.decode(s).replace(' ', '')
        for s in ret.numpy()
    ]

# 问答例子

In [9]:
def ask_gpt(question):
    q = f'''
问题：中国首都是哪里？
答案：北京
问题：美国的首都是哪里？
答案：华盛顿
问题：李白在哪个朝代？
答案：唐朝
问题：{question}
答案：'''
    ret = sample(tokenizer, gpt, q, 3, 10, top_p=0.9, temperature=0.9)
    answers = []
    for x in ret:
        a = x[len(q):]
        a = a.split('\n')[0]
        answers.append(a)
    return answers

In [10]:
ask_gpt('唐伯虎是哪个朝代的人？')

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.679 seconds.
DEBUG:jieba:Loading model cost 0.679 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


['明代', '明朝', '唐伯虎']

In [11]:
ask_gpt('世界上最帅的是谁？')

['贝克汉姆', '张国荣', '普京']

In [12]:
ask_gpt('世界上最聪明的人是谁？')

['孔子', '霍金', '乔达摩·悉达多']

In [13]:
ask_gpt('李清照是哪个朝代的人？')

['宋朝', '北宋', '北宋']

In [14]:
ask_gpt('关公和秦琼谁厉害？')

['关公', '关羽和尉迟恭谁厉害?', '关公']

In [15]:
ask_gpt('如何赚一百万美元？')

['开始投资股票', '办报纸', '卖衣服']

In [16]:
ask_gpt('烧烤和火锅哪个好吃？')

['中餐', '都好吃', '海鲜和烤串']

In [17]:
ask_gpt('如何成为老师？')

['教“国”', '孔子', '帮助孩子成为一个最好的老师']

In [18]:
ask_gpt('佩奇是什么动物？')

['鲸鱼', '海獭', '羚羊']

In [19]:
ask_gpt('大象有几条腿？')

['两条腿', '三条腿', '两条腿']

In [20]:
ask_gpt('姚明有多高？')

['NBA球员迈克尔·乔丹', '6英尺10英寸', '姚明']

In [21]:
ask_gpt('汤姆和杰瑞谁厉害？')

['汤姆', '谁强谁弱?', '汤姆']

In [22]:
ask_gpt('野原新之助能打过樱桃子吗？')

['樱桃子是美国人的绰号', '不知道', '不知道']

In [23]:
ask_gpt('学音乐和学画画哪个好？')

['学好了都有出路', '都不好', '不好说']

In [24]:
ask_gpt('中文和英文哪个难学？')

['中文', '中文', '英语']

In [25]:
ask_gpt('python和java哪个难？')

['python', 'Python', 'Java']

In [26]:
ask_gpt('“锄禾日当午”的下一句是什么？')

['“汗滴禾下土”', '“谁知盘中餐,粒粒皆', '“谁知盘中餐,粒粒皆']

In [27]:
ask_gpt('烤鸭的主要原料是什么？')

['鸭', '鸭肉', '鸭肉']

In [28]:
ask_gpt('把大象放冰箱总共分几步？')

['从最容易的开始', '把大象放冰箱', '先放进冰箱,再放进大象']

In [29]:
ask_gpt('珠穆朗玛峰有多高？')

['8848米', '6300米', '8844米']

In [30]:
ask_gpt('世界上最高的山是什么？')

['珠穆朗玛峰', '珠穆朗玛峰', '珠穆朗玛峰']

In [31]:
ask_gpt('中国有多少人？')

['一亿多人', '103万', '3亿']

In [32]:
ask_gpt('日本有多少人？')

['0', '人数在160万以上', '672万人']

In [33]:
ask_gpt('苹果手机好还是华为手机好？')

['小米手机', '苹果', '苹果']

# 推理

In [34]:
ret = sample(tokenizer, gpt, '小明决定去吃饭，小红继续写作业\n问题：去吃饭的人是谁？\n答案：', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小红、小明、小丽、小红
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:小明和小红
整理:小明
--------------------
小明决定去吃饭,小红继续写作业
问题:去吃饭的人是谁?
答案:一个平时不怎么讲话的女生
问题:
--------------------


# 英文问答例子

In [7]:
ret = sample(tokenizer, gpt, '默写英文：\n狗dog\n猫cat\n鸟', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写英文:
狗dog
猫cat
鸟bird
熊earth

--------------------
默写英文:
狗dog
猫cat
鸟birds
爱丽丝alove
--------------------
默写英文:
狗dog
猫cat
鸟bird
船boat

--------------------


# 默写古诗例子

In [8]:
ret = sample(tokenizer, gpt, '默写古诗：\n白日依山尽，黄河入海流。\n床前明月光，', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------
默写古诗:
白日依山尽,黄河入海流。
床前明月光,疑是地上霜。
举头望
--------------------


# 不同的角色对话生成例子

In [9]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n佟掌柜：“', 3, 20, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
佟掌柜:“怎么着,想打架?”
江白:“不
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“说什么呢,哪有这么吃的?”
展昭:
--------------------
李大嘴:“各回各家,各找各妈!”
佟掌柜:“我没听见这几个字。”
柳大洪:
--------------------


In [10]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n白展堂：“', 3, 20, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
白展堂:“我要给你生个孩子!”
“孩子!”
--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“不是,是咱‘一言为定’!”

--------------------
李大嘴:“各回各家,各找各妈!”
白展堂:“嗯,嗯,好,我回家了。”
--------------------


In [11]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n莫小贝：“', 3, 20, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
莫小贝:“现在别叫了,我想起来了,知道,知道!
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“你个死鬼,敢骗我!”
沈浪:
--------------------
李大嘴:“各回各家,各找各妈!”
莫小贝:“他好像也喜欢我。”
徐天:“
--------------------


In [12]:
ret = sample(tokenizer, gpt, '李大嘴：“各回各家，各找各妈！” \n李白：“', 3, 20, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李大嘴:“各回各家,各找各妈!”
李白:“阿舅,你真不是人,我要去找我妈,你快拦
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“啊!那就一言为定,我走我的阳关道,你
--------------------
李大嘴:“各回各家,各找各妈!”
李白:“我没文化,我有文化,我是文化人!”

--------------------


# 问答的例子

In [13]:
ret = sample(tokenizer, gpt, '中国的首都是北京\n日本的首都是东京\n美国的首都是', 3, 3, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
俄国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
英国
--------------------
中国的首都是北京
日本的首都是东京
美国的首都是华盛顿
朝鲜
--------------------


In [15]:
ret = sample(tokenizer, gpt, '李白所在朝代是唐\n李清照所在朝代是宋\n唐伯虎所在朝代是', 3, 1, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是元
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------
李白所在朝代是唐
李清照所在朝代是宋
唐伯虎所在朝代是明
--------------------


In [63]:
ret = sample(tokenizer, gpt, '陈奕迅是歌星\n郭德纲是', 3, 1, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

陈奕迅是歌星
郭德纲是相声演员
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------
陈奕迅是歌星
郭德纲是相声演员
--------------------


In [68]:
ret = sample(tokenizer, gpt, '珠穆朗玛峰的高度（米）是', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

珠穆朗玛峰的高度(米)是8351。
--------------------
珠穆朗玛峰的高度(米)是1432.786。<eod>诗词:新
--------------------
珠穆朗玛峰的高度(米)是多少?(如果和世界最高峰珠
--------------------


In [67]:
ret = sample(tokenizer, gpt, '姚明的身高（米）是', 3, 10, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

姚明的身高(米)是六英尺四英寸,因为
--------------------
姚明的身高(米)是多少?请分别说明姚明的身高
--------------------
姚明的身高(米)是NBA球员中最高的,所以是中
--------------------


# 算数例子

In [16]:
ret = sample(tokenizer, gpt, '1+1=2\n2+2=4\n3+3=6\n4+4=', 3, 1, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
2+2=4
3+3=6
4+4=8
--------------------
1+1=2
2+2=4
3+3=6
4+4=8
--------------------
1+1=2
2+2=4
3+3=6
4+4=12
--------------------


In [17]:
ret = sample(tokenizer, gpt, '1+1=2\n1+2=3\n1+3=4\n1+4=', 3, 1, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------
1+1=2
1+2=3
1+3=4
1+4=5
--------------------


# ???的例子

In [14]:
ret = sample(tokenizer, gpt, '''惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
''', 3, 30, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
天崩地裂紫金锤
天昏地暗紫金锤
三魂被震散
紫电被震散
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
冲天三式斩雷斧
幻影这紫电玄焰
九龙金锤
且看圣天之上斩
--------------------
惊雷这通天修为
天塌地陷紫金锤
紫电这玄真火焰
玄罡这丹药
玄真这烈火
玄真这丹药
仙灵没利爪
玄真没
--------------------


# 写作文例子

In [18]:
ret = sample(tokenizer, gpt, '''爱情是''', 3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱情是一定要三观一致的,不然就算再相爱也会觉得累。但我男朋友要出国读研,和他在一起我一直都觉得要么我和他在一起,要
--------------------
爱情是美好的,但是婚姻不一定美好。现实很残酷,人心很虚,这一点我很清楚。我在做自己想做的事情,不伤害别人,也不委屈自己。
--------------------
爱情是付出,你我彼此互相爱,互相依赖,互相包容,互相支持,共同努力,共同承担,遇到困难,想到对方,想到对方的好,自己的不好
--------------------


In [19]:
ret = sample(
    tokenizer, gpt,
    '''一时黛玉进了荣府，下了车。众嬷嬷引着，便往东转弯，穿过一个东西的穿堂，向南大厅之后，仪门内大院落，上面五间大正房，两边厢房鹿顶耳房钻山，四通八达，轩昂壮丽，比贾母处不同。黛玉便知这方是正经正内室，一条大甬路，直接出大门的。''',
    3, 200, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。方家的人亦进不来,似乎贾母房中的众姊妹们原封不动带进来,所以知道。
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。这贾母王夫人等,不是宝玉等,也不是老太太王夫人,却是姨太太的丈夫,姨太太的妻子,宝玉等不过是他们的小辈,他们的妻妾。黛玉见了这般,心中更觉惊奇,只是笑道:“姨太太住的是咱们这边正房,我是荣国府的人,不便进去的。”说着,便走至贾母那边,拉着贾母,只说要去,贾母也没料到她会来,便道:“我那边没人,是我小老婆的丈夫,他们一家子搬到这边来住了。”宝玉笑道:“原来你们那边也有
--------------------
一时黛玉进了荣府,下了车。众嬷嬷引着,便往东转弯,穿过一个东西的穿堂,向南大厅之后,仪门内大院落,上面五间大正房,两边厢房鹿顶耳房钻山,四通八达,轩昂壮丽,比贾母处不同。黛玉便知这方是正经正内室,一条大甬路,直接出大门的。再回头看看贾母,忙忙的张罗着自己收拾屋子,桌上放着大笔银子,舅母宝玉那里送来的。这里我们可以看到,贾母住的大房子的房顶是四角的,其中靠外的两角是院子,住的是贾母,住的是二房的贾敏,贾敏住的是正房的三间,贾母住的是外间,四间有大花的厅子,五间有小花的厅子,住的是丫头媳妇,其中三间住的是宝玉,贾母住的是正房的三间,贾敏住的是外间,在这里,小花厅的住的是鸳鸯,内间住的是林黛玉。这还不算
--------------------


# 对话例子

In [20]:
ret = sample(
    tokenizer, gpt,
    '''A：“今天我想吃火锅”
B：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“今天我想吃火锅”
B:“你想吃什么火锅?”
A:“火锅”
B:“哦,那吃鱼吧”
A:“鱼”
B:“
--------------------
A:“今天我想吃火锅”
B:“哦,那我知道了”
A:“咱们一起去吃火锅吧”
B:“还是算了吧,我吃过了”
A
--------------------
A:“今天我想吃火锅”
B:“我想吃烤鱼”
C:“我想吃烤鸭”
D:“我想吃鸡排”
E:“我想吃日式拉面”
--------------------


In [27]:
ret = sample(
    tokenizer, gpt, '''A：“跟我一起去看电影吧”
B：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

A:“跟我一起去看电影吧”
B:“好啊”
A:“看什么电影?”
B:“《贞子3D》”
A:“原来你是SHE的歌迷,
--------------------
A:“跟我一起去看电影吧”
B:“嗯,好”
A:“去电影院吗?”
B:“是的”
A:“你不是单身吗?”
B:
--------------------
A:“跟我一起去看电影吧”
B:“好啊”
A:“其实我还不知道你是谁,你叫什么名字?”
B:“(⊙o⊙)...”
A
--------------------


# 对联例子

In [49]:
duilian = '''
上联天，下联地
上联雨，下联风
上联大陆，下联长空
上联雷隐隐，下联雾蒙蒙
上联开心大吉，下联万事亨通
'''

In [56]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联李白作诗，下联',
    5, 2, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫作诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联李白作诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫作诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联杜甫作诗
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联李白作诗,下联李白作赋
--------------------


In [55]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联追求真爱，下联',
    5, 4, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联只要有福
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联拒绝早恋

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联吃牢饭
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联幸福一生
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联追求真爱,下联享尽富贵

--------------------


In [54]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天天向上，下联',
    5, 4, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联白日青天
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联天天向上
上
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联长命百岁
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联平平安安

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天天向上,下联夜夜同欢
--------------------


In [58]:
ret = sample(
    tokenizer, gpt,
    duilian + '上联天地在我心，下联',
    5, 4, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)


上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联万事皆休

--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联风雪在我怀
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联我心在天地
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联天地都是家
--------------------

上联天,下联地
上联雨,下联风
上联大陆,下联长空
上联雷隐隐,下联雾蒙蒙
上联开心大吉,下联万事亨通
上联天地在我心,下联地久天长
--------------------


# 名人名言

In [30]:
ret = sample(
    tokenizer, gpt, '''鲁迅曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

鲁迅曾经说过:“问题不在主义,而在于主义之下的中国社会。”面对外国的“新文化运动”,我们该如何对待呢?胡适、鲁迅
--------------------
鲁迅曾经说过:“中国人的性情是总喜欢调和、折中的。譬如你说,这屋子太暗,说在这里开一个天窗,大家一定是不允许的。但如果你主张拆
--------------------
鲁迅曾经说过:“志摩是个乐观主义者,他总是以开朗的笑容面对生活,即使碰上了不顺心的事,他也很快会抛到脑后。”3
--------------------


In [31]:
ret = sample(
    tokenizer, gpt, '''爱因斯坦曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

爱因斯坦曾经说过:“唯物主义者,只有唯物主义者,才可称得上哲学家。”唯物主义也是无神论,它从宗教的宗教立场出发,研究自然和人类社会的规律,如人
--------------------
爱因斯坦曾经说过:“科学的尽头是哲学,哲学的尽头是宗教。”爱因斯坦说这句话的意思是,如果你试图用一个方法来解决一个问题,那么你的最终目标肯定是错误
--------------------
爱因斯坦曾经说过:“在我们的发展过程中,不断学习,因为学习的成果是不可估量的,我们应该不断地从别人的研究成果中吸取营养,就像纳豆、大枣、山药
--------------------


In [32]:
ret = sample(
    tokenizer, gpt, '''牛顿曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

牛顿曾经说过:“在自然科学的历史上,许多伟大的人物,都是在受了科学家的启发之后,才发奋地去从事研究的。”一个伟大的科学家,对于科学家的
--------------------
牛顿曾经说过:“凡能够称为哲学的,都可以称为科学。”这一点早就被否定了。现在流行的哲学流派,大部分都不是科学。我并不反对
--------------------
牛顿曾经说过:“我一生的科学工作就是把真理变成人类能够懂得的语言。”科学不仅仅是一个探索真理的过程,还是人类探索真理的一条捷径。我们每个人
--------------------


In [33]:
ret = sample(
    tokenizer, gpt, '''佛祖曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

佛祖曾经说过:“人皆可以为尧舜,但我不可以。”所以王阳明到底还是失败了。这让王阳明在内心颇为沮丧,同时又无法释怀。
--------------------
佛祖曾经说过:“我给你一颗痣,你要还给我。”唐僧听到,慌忙跪在地上,问佛祖:“你给我一颗痣,我也要还给你吗
--------------------
佛祖曾经说过:“如果一个人说他没有钱,那不是很没面子吗?”这话我是赞同的,谁都会这么说,但我想说的是,如果一个人在说这句话的
--------------------


In [34]:
ret = sample(
    tokenizer, gpt, '''乔布斯曾经说过：“''',
    3, 50, top_p=0.9, temperature=0.9)
for x in ret:
    print(x)
    print('-' * 20)

乔布斯曾经说过:“如果你不理解为什么会有人这么做,那么你就没必要去理解。如果你真的想理解为什么会有人这么做,那么你就得先去理解。”—
--------------------
乔布斯曾经说过:“你不会用过了,再来后悔。”“不要为打翻的牛奶哭泣”(教父)我
--------------------
乔布斯曾经说过:“我们在我认为是伟大产品中,并不看重它的功能,我们看重的是是否解决了问题。”那么你有没有发现,“解决问题”这个前提
--------------------
